In [73]:
# The function localize takes the following arguments:
#
# colors:
#        2D list, each entry either 'R' (for red cell) or 'G' (for green cell)
#
# measurements:
#        list of measurements taken by the robot, each entry either 'R' or 'G'
#
# motions:
#        list of actions taken by the robot, each entry of the form [dy,dx],
#        where dx refers to the change in the x-direction (positive meaning
#        movement to the right) and dy refers to the change in the y-direction
#        (positive meaning movement downward)
#        NOTE: the *first* coordinate is change in y; the *second* coordinate is
#              change in x
#
# sensor_right:
#        float between 0 and 1, giving the probability that any given
#        measurement is correct; the probability that the measurement is
#        incorrect is 1-sensor_right
#
# p_move:
#        float between 0 and 1, giving the probability that any given movement
#        command takes place; the probability that the movement command fails
#        (and the robot remains still) is 1-p_move; the robot will NOT overshoot
#        its destination in this exercise
#
# The function should RETURN (not just show or print) a 2D list (of the same
# dimensions as colors) that gives the probabilities that the robot occupies
# each cell in the world.
#
# Compute the probabilities by assuming the robot initially has a uniform
# probability of being in any cell.
#
# Also assume that at each step, the robot:
# 1) first makes a movement,
# 2) then takes a measurement.
#
# Motion:
#  [0,0] - stay
#  [0,1] - right
#  [0,-1] - left
#  [1,0] - down
#  [-1,0] - up

def isRight(move):
    return move[0] == 0 and move[1] == 1

def isLeft(move):
    return move[0] == 0 and move[1] == -1

def isDown(move):
    return move[0] == 1 and move[1] == 0

def isUp(move):
    return move[0] == -1 and move[1] == 0

def isHit(real, measured):
    return real == measured

def senseRow(realRow, probabilityRow, measurement, p_correct):
    res = probabilityRow[:]
    
    for index in range(len(probabilityRow)):
        res[index] *= p_correct if isHit(realRow[index], measurement) else (1 - p_correct)
    
    return res

def normalize(matrix):
    s = sum([sum(row) for row in matrix])
    normalized = [[elem / s for elem in row] for row in matrix]
    
    return normalized

def moveRight(pRow, p_move):
    res = pRow[:]
    for i,v in enumerate(pRow):
        res[i] = pRow[i] * (1 - p_move) + pRow[i-1] * p_move
    
    return res

def moveLeft(pRow, p_move):
    res = pRow[:]
    
    for i, value in enumerate(pRow):
        res[i] = pRow[i] * (1 - p_move) + pRow[(i + 1) % len(pRow)] * p_move
    
    return res

def transpose(matrix):
    return [list(i) for i in zip(*matrix)]

def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print('[' + ',\n '.join(rows) + ']')

def localize(colors, measurements, motions, sensor_right, p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    for i in range(len(motions)):
        if isRight(motions[i]):
            for rowIndex in range(len(p)):
                p[rowIndex] = moveRight(p[rowIndex], p_move)
        elif isLeft(motions[i]):
            for rowIndex in range(len(p)):
                p[rowIndex] = moveLeft(p[rowIndex], p_move)
        elif isDown(motions[i]):
            p = transpose(p)
            for rowIndex in range(len(p)):
                p[rowIndex] = moveRight(p[rowIndex], p_move)
            p = transpose(p)
        elif isUp(motions[i]):
            p = transpose(p)
            for rowIndex in range(len(p)):
                p[rowIndex] = moveLeft(p[rowIndex], p_move)
            p = transpose(p)
        else:
            pass
        
        for rowIndex in range(len(p)):
            p[rowIndex] = senseRow(colors[rowIndex], p[rowIndex], measurements[i], sensor_right)
        
        p = normalize(p)
           
    return p


    
#############################################################
# For the following test case, your output should be 
# [[0.01105, 0.02464, 0.06799, 0.04472, 0.02465],
#  [0.00715, 0.01017, 0.08696, 0.07988, 0.00935],
#  [0.00739, 0.00894, 0.11272, 0.35350, 0.04065],
#  [0.00910, 0.00715, 0.01434, 0.04313, 0.03642]]
# (within a tolerance of +/- 0.001 for each entry)

colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)


show(p) # displays your answer

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]
